# Segmenting and Clustering Visualization Notebook

### Create Database using previous notebook

In [1]:
import pandas as pd
import requests
!conda install -c conda-forge beautifulsoup4  --yes  #Install BS4 if environment does not have installed
from bs4 import BeautifulSoup
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,features='html')

#print(soup.prettify())
post_table = soup.find('table',{'class':'wikitable sortable'})

pcode = []
bor = []
neigh = []

rows = post_table.find_all('tr')
    
for row in rows:
    cells = row.find_all('td')
        
        
    if len(cells) > 1:
          
        if cells[1].text.strip()== 'Not assigned':
             next 
        else:
            c0 = cells[0].text.strip()
            pcode.append(c0)
            
            c1 = cells[1].text.strip()
            bor.append(c1)
            
            c2 = cells[2].text.strip()
               
            if c2 == 'Not assigned':    
              neigh.append(c1)
            else:
              neigh.append(c2)
            
wiki = pd.DataFrame({'PostalCode':pcode,'Borough':bor,'Neighbourhood':neigh})

geo = pd.read_csv('http://cocl.us/Geospatial_data')

wiki_t = wiki.merge(geo.rename(columns={"Postal Code":"PostalCode"}),on="PostalCode")
wiki_1 = wiki.groupby('PostalCode')['Neighbourhood'].apply(list).to_frame()
pc1 =[]
b1=[]
count = 0
for i in wiki_t['PostalCode']:
    if i not in pc1:
        pc1.append(i)
        b1.append(wiki_t['Borough'][count])
        count = count + 1
        
df1 = pd.DataFrame({'PostalCode':pc1,'Borough':b1})
df2 = wiki_1.merge(geo.rename(columns={"Postal Code":"PostalCode"}),on="PostalCode")
df3 = df1.merge(df2,on="PostalCode")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forg

#### Install Geopy and Folium for geographical and mapping techniques

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 3

### Create Toronto Map with neighbourhood and postcode markers

In [5]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, pc2, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'],
                                           df3['PostalCode'],df3['Borough'],
                                           df3['Neighbourhood']):
    label = '{}, {}'.format(pc2, neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor